In [1]:
import numpy as np
import os
import numpy
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel as C



In [2]:
## Helper function

''' Converts black box functions from np.arrays to pandas.DataFrames for easier interpretation of tabular data
Inputs:
    npa_x       inputs x1,x2,... as np array
    npa_y       output y as np array

Outputs:
    df          Dataframe with columns x1, x2, ... and y
'''

def fcn_as_df(npa_x,npa_y):
    # assure that even single rows have two dimensions - avoids breaking code below
    if npa_x.ndim == 1:
        npa_x = npa_x.reshape(1,-1)
        npa_y = npa_y.reshape(1,-1)

    n_col = npa_x.shape[1]
    col_names = [f"x{i+1}" for i in range(n_col)]
    df = pd.DataFrame(npa_x,columns=col_names)
    df["y"] = np.ravel(npa_y).T
    return df

''' Helper - Reads numpy arrays for each of the eight functions from disk and returns a list of dataframes containing the input and output info

Inputs: 
    path_to_fcns    Directory path - must contain folders "function_1","function_2"... each containing inputs.npy and outputs.npy 

Outputs: 
    fcn_dict - Dictionary of dataframes for each function, each dataframe containing columns x1, x2,..., y
'''


def read_fcns_from_disk(path_to_fcns):
    f_in = []
    f_out = []
    num_functions = 8
    # read the functions as a list of numpy arrays, separate for inputs x and output y
    for i in range(0, num_functions):
        dir_path = os.path.join(path_to_fcns, f'function_{i+1}')
        inputs = np.load(os.path.join(dir_path, 'inputs.npy'))
        outputs = np.load(os.path.join(dir_path, 'outputs.npy'))
        f_in.append(inputs)
        f_out.append(outputs)
        
    # create a dictionary of form {"f_1": DataFrame, "f_2": DataFrame, } where the DataFrames have column names x1, x2, ..., y
    fcn_dict = {}
    for i in range(0, num_functions):
        npa_x = f_in[i]
        npa_y = f_out[i]

        fcn_dict[f"f_{i+1}"] = fcn_as_df(npa_x,npa_y)

    return fcn_dict




In [3]:

latest_data_dir = os.path.join(".","measurements","latest")
f_dict = read_fcns_from_disk(latest_data_dir)

for k in f_dict.keys():
    print(k)
    print("---")
    print(f_dict[k])



f_1
---
          x1        x2              y
0   0.319404  0.762959   1.322677e-79
1   0.574329  0.879898   1.033078e-46
2   0.731024  0.733000   7.710875e-16
3   0.840353  0.264732  3.341771e-124
4   0.650114  0.681526  -3.606063e-03
5   0.410437  0.147554  -2.159249e-54
6   0.312691  0.078723  -2.089093e-91
7   0.683418  0.861057   2.535001e-40
8   0.082507  0.403488   3.606771e-81
9   0.883890  0.582254   6.229856e-48
10  0.997304  1.000000  3.215893e-191
f_2
---
          x1        x2         y
0   0.665800  0.123969  0.538996
1   0.877791  0.778628  0.420586
2   0.142699  0.349005 -0.065624
3   0.845275  0.711120  0.293993
4   0.454647  0.290455  0.214965
5   0.577713  0.771973  0.023106
6   0.438166  0.685018  0.244619
7   0.341750  0.028698  0.038749
8   0.338648  0.213867 -0.013858
9   0.702637  0.926564  0.611205
10  0.758325  1.000000  0.240588
f_3
---
          x1        x2        x3         y
0   0.171525  0.343917  0.248737 -0.112122
1   0.242114  0.644074  0.272433 -0.08

In [4]:
# Helper function to print query point in expected format

def format_query(queryPoint):
    
    print(f"queryPoint={queryPoint}")

    # print the elements of the vector queryPoint, in the format 0.000000-0.000000-0.000000
    # where there is one 0.000000 per element in the vector

    query_str = '-'.join([f"{elem:.6f}" for elem in np.ravel(queryPoint)])
    print(f"queryPoint (formatted)={query_str}")

    #query_str=""
    #query_str.join(f"{queryPoint[0]:.6f}")
    #query_str.join([f"{elem:.6f}" for elem in np.ravel(queryPoint[1:])])
    #print(f"queryPoint (formatted)={query_str}")

In [5]:
# fit RBF to f1 AND SEARCH FOR MAX OF THE LOG!!!!!!!!

# pick out function 1
k = "f_1"
f = f_dict[k]

y_train = f.iloc[:,-1].to_numpy()  # output is in last column
X_train = f.iloc[:,:-1].to_numpy() # other columns are inputs

np.random.seed(0)
kernel = C(1.0, (1e-3, 1e3)) * Matern(length_scale=[1.0, 1.0], nu=2.5) + WhiteKernel(noise_level=1e-5)
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5, normalize_y=True)
gp.fit(X_train, y_train)

x1 = np.linspace(0, 1, 50)
x2 = np.linspace(0, 1, 50)
X1, X2 = np.meshgrid(x1, x2)
X_grid = np.vstack((X1.ravel(), X2.ravel())).T

# --- Step 5. Predict mean and std from the GP ---
y_mean, y_std = gp.predict(X_grid, return_std=True)

# --- Step 6. Define the UCB (Upper Confidence Bound) acquisition function ---
beta = 2.0  # exploration parameter
ucb = y_mean + beta * y_std

# --- Step 7. Select next acquisition point ---
next_idx = np.argmax(ucb)
X_next = X_grid[next_idx]
print("Next acquisition point (UCB):", X_next)

format_query(X_next)


Next acquisition point (UCB): [0.65306122 0.20408163]
queryPoint=[0.65306122 0.20408163]
queryPoint (formatted)=0.653061-0.204082


/Users/muchansk/repo/mlcap/.venv/lib/python3.13/site-packages/sklearn/gaussian_process/kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


In [6]:
# fit RBF to f2

# pick out function 2
k = "f_2"
f = f_dict[k]

y_train = f.iloc[:,-1].to_numpy()  # output is in last column
X_train = f.iloc[:,:-1].to_numpy() # other columns are inputs

np.random.seed(0)
kernel = C(1.0, (1e-3, 1e3)) * Matern(length_scale=[1.0, 1.0], nu=2.5) + WhiteKernel(noise_level=1e-5)
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5, normalize_y=True)
gp.fit(X_train, y_train)

x1 = np.linspace(0, 1, 50)
x2 = np.linspace(0, 1, 50)
X1, X2 = np.meshgrid(x1, x2)
X_grid = np.vstack((X1.ravel(), X2.ravel())).T

# --- Step 5. Predict mean and std from the GP ---
y_mean, y_std = gp.predict(X_grid, return_std=True)

# --- Step 6. Define the UCB (Upper Confidence Bound) acquisition function ---
beta = 2.0  # exploration parameter
ucb = y_mean + beta * y_std

# --- Step 7. Select next acquisition point ---
next_idx = np.argmax(ucb)
X_next = X_grid[next_idx]
print("Next acquisition point (UCB):", X_next)

format_query(X_next)


Next acquisition point (UCB): [0. 0.]
queryPoint=[0. 0.]
queryPoint (formatted)=0.000000-0.000000


In [7]:
# fit RBF to f3

# pick out function 3
k = "f_3"
f = f_dict[k]

y_train = f.iloc[:,-1].to_numpy()  # output is in last column
X_train = f.iloc[:,:-1].to_numpy() # other columns are inputs

np.random.seed(0)
kernel = C(1.0, (1e-3, 1e3)) * Matern(length_scale=[1.0, 1.0, 1.0], nu=2.5) + WhiteKernel(noise_level=1e-5)
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5, normalize_y=True)
gp.fit(X_train, y_train)

x1 = np.linspace(0, 1, 50)
x2 = np.linspace(0, 1, 50)
x3 = np.linspace(0, 1, 50)
X1, X2, X3 = np.meshgrid(x1, x2, x3)
X_grid = np.vstack((X1.ravel(), X2.ravel(), X3.ravel())).T

# --- Step 5. Predict mean and std from the GP ---
y_mean, y_std = gp.predict(X_grid, return_std=True)

# --- Step 6. Define the UCB (Upper Confidence Bound) acquisition function ---
beta = 2.0  # exploration parameter
ucb = y_mean + beta * y_std

# --- Step 7. Select next acquisition point ---
next_idx = np.argmax(ucb)
X_next = X_grid[next_idx]
print("Next acquisition point (UCB):", X_next)

format_query(X_next)

/Users/muchansk/repo/mlcap/.venv/lib/python3.13/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 1 of parameter k1__k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


Next acquisition point (UCB): [0. 0. 0.]
queryPoint=[0. 0. 0.]
queryPoint (formatted)=0.000000-0.000000-0.000000


In [8]:
# fit RBF to f4

# pick out function 4
k = "f_4"
f = f_dict[k]

y_train = f.iloc[:,-1].to_numpy()  # output is in last column
X_train = f.iloc[:,:-1].to_numpy() # other columns are inputs

np.random.seed(0)
kernel = C(1.0, (1e-3, 1e3)) * Matern(length_scale=[1.0, 1.0, 1.0, 1.0], nu=2.5) + WhiteKernel(noise_level=1e-5)
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5, normalize_y=True)
gp.fit(X_train, y_train)

x1 = np.linspace(0, 1, 50)
x2 = np.linspace(0, 1, 50)
x3 = np.linspace(0, 1, 50)
x4 = np.linspace(0, 1, 50)
X1, X2, X3, X4 = np.meshgrid(x1, x2, x3, x4)
X_grid = np.vstack((X1.ravel(), X2.ravel(), X3.ravel(), X4.ravel())).T

# --- Step 5. Predict mean and std from the GP ---
y_mean, y_std = gp.predict(X_grid, return_std=True)

# --- Step 6. Define the UCB (Upper Confidence Bound) acquisition function ---
beta = 2.0  # exploration parameter
ucb = y_mean + beta * y_std

# --- Step 7. Select next acquisition point ---
next_idx = np.argmax(ucb)
X_next = X_grid[next_idx]
print("Next acquisition point (UCB):", X_next)

format_query(X_next)

/Users/muchansk/repo/mlcap/.venv/lib/python3.13/site-packages/sklearn/gaussian_process/kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


Next acquisition point (UCB): [0.40816327 0.40816327 0.36734694 0.42857143]
queryPoint=[0.40816327 0.40816327 0.36734694 0.42857143]
queryPoint (formatted)=0.408163-0.408163-0.367347-0.428571


In [9]:
# fit RBF to f5

# pick out function 5
k = "f_5"
f = f_dict[k]

y_train = f.iloc[:,-1].to_numpy()  # output is in last column
X_train = f.iloc[:,:-1].to_numpy() # other columns are inputs

np.random.seed(0)
kernel = C(1.0, (1e-3, 1e3)) * Matern(length_scale=[1.0, 1.0, 1.0, 1.0], nu=2.5) + WhiteKernel(noise_level=1e-5)
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5, normalize_y=True)
gp.fit(X_train, y_train)

x1 = np.linspace(0, 1, 50)
x2 = np.linspace(0, 1, 50)
x3 = np.linspace(0, 1, 50)
x4 = np.linspace(0, 1, 50)
X1, X2, X3, X4 = np.meshgrid(x1, x2, x3, x4)
X_grid = np.vstack((X1.ravel(), X2.ravel(), X3.ravel(), X4.ravel())).T

# --- Step 5. Predict mean and std from the GP ---
y_mean, y_std = gp.predict(X_grid, return_std=True)

# --- Step 6. Define the UCB (Upper Confidence Bound) acquisition function ---
beta = 2.0  # exploration parameter
ucb = y_mean + beta * y_std

# --- Step 7. Select next acquisition point ---
next_idx = np.argmax(ucb)
X_next = X_grid[next_idx]
print("Next acquisition point (UCB):", X_next)

format_query(X_next)

/Users/muchansk/repo/mlcap/.venv/lib/python3.13/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/muchansk/repo/mlcap/.venv/lib/python3.13/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 1 of parameter k1__k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/muchansk/repo/mlcap/.venv/lib/python3.13/site-packages/sklearn/gaussian_process/kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


Next acquisition point (UCB): [1. 0. 1. 1.]
queryPoint=[1. 0. 1. 1.]
queryPoint (formatted)=1.000000-0.000000-1.000000-1.000000


In [10]:
# fit RBF to f6

# pick out function 6
k = "f_6"
f = f_dict[k]

y_train = f.iloc[:,-1].to_numpy()  # output is in last column
X_train = f.iloc[:,:-1].to_numpy() # other columns are inputs

np.random.seed(0)
kernel = C(1.0, (1e-3, 1e3)) * Matern(length_scale=[1.0, 1.0, 1.0, 1.0, 1.0], nu=2.5) + WhiteKernel(noise_level=1e-5)
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5, normalize_y=True)
gp.fit(X_train, y_train)

x1 = np.linspace(0, 1, 20)
x2 = np.linspace(0, 1, 20)
x3 = np.linspace(0, 1, 20)
x4 = np.linspace(0, 1, 20)
x5 = np.linspace(0, 1, 20)
X1, X2, X3, X4, X5 = np.meshgrid(x1, x2, x3, x4, x5)
X_grid = np.vstack((X1.ravel(), X2.ravel(), X3.ravel(), X4.ravel(), X5.ravel())).T

# --- Step 5. Predict mean and std from the GP ---
y_mean, y_std = gp.predict(X_grid, return_std=True)

# --- Step 6. Define the UCB (Upper Confidence Bound) acquisition function ---
beta = 2.0  # exploration parameter
ucb = y_mean + beta * y_std

# --- Step 7. Select next acquisition point ---
next_idx = np.argmax(ucb)
X_next = X_grid[next_idx]
print("Next acquisition point (UCB):", X_next)

format_query(X_next)

Next acquisition point (UCB): [0.31578947 0.05263158 0.57894737 1.         0.        ]
queryPoint=[0.31578947 0.05263158 0.57894737 1.         0.        ]
queryPoint (formatted)=0.315789-0.052632-0.578947-1.000000-0.000000


In [11]:
# fit RBF to f7

# pick out function 7
k = "f_7"
f = f_dict[k]

y_train = f.iloc[:,-1].to_numpy()  # output is in last column
X_train = f.iloc[:,:-1].to_numpy() # other columns are inputs

np.random.seed(0)
kernel = C(1.0, (1e-3, 1e3)) * Matern(length_scale=[1.0, 1.0, 1.0, 1.0, 1.0, 1.0], nu=2.5) + WhiteKernel(noise_level=1e-5)
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5, normalize_y=True)
gp.fit(X_train, y_train)

x1 = np.linspace(0, 1, 10)
x2 = np.linspace(0, 1, 10)
x3 = np.linspace(0, 1, 10)
x4 = np.linspace(0, 1, 10)
x5 = np.linspace(0, 1, 10)
x6 = np.linspace(0, 1, 10)

X1, X2, X3, X4, X5, X6 = np.meshgrid(x1, x2, x3, x4, x5, x6)
X_grid = np.vstack((X1.ravel(), X2.ravel(), X3.ravel(), X4.ravel(), X5.ravel(), X6.ravel())).T

# --- Step 5. Predict mean and std from the GP ---
y_mean, y_std = gp.predict(X_grid, return_std=True)

# --- Step 6. Define the UCB (Upper Confidence Bound) acquisition function ---
beta = 2.0  # exploration parameter
ucb = y_mean + beta * y_std

# --- Step 7. Select next acquisition point ---
next_idx = np.argmax(ucb)
X_next = X_grid[next_idx]
print("Next acquisition point (UCB):", X_next)

format_query(X_next)

/Users/muchansk/repo/mlcap/.venv/lib/python3.13/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/muchansk/repo/mlcap/.venv/lib/python3.13/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 1 of parameter k1__k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/muchansk/repo/mlcap/.venv/lib/python3.13/site-packages/sklearn/gaussian_process/kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


Next acquisition point (UCB): [0.         0.         0.         0.55555556 0.         0.33333333]
queryPoint=[0.         0.         0.         0.55555556 0.         0.33333333]
queryPoint (formatted)=0.000000-0.000000-0.000000-0.555556-0.000000-0.333333


In [ ]:
# fit RBF to f8

# pick out function 8
k = "f_8"
f = f_dict[k]

y_train = f.iloc[:,-1].to_numpy()  # output is in last column
X_train = f.iloc[:,:-1].to_numpy() # other columns are inputs

np.random.seed(0)
kernel = C(1.0, (1e-3, 1e3)) * Matern(length_scale=[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], nu=2.5) + WhiteKernel(noise_level=1e-5)
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5, normalize_y=True)
gp.fit(X_train, y_train)

x1 = np.linspace(0, 1, 5)
x2 = np.linspace(0, 1, 5)
x3 = np.linspace(0, 1, 5)
x4 = np.linspace(0, 1, 5)
x5 = np.linspace(0, 1, 5)
x6 = np.linspace(0, 1, 5)
x7 = np.linspace(0, 1, 5)
x8 = np.linspace(0, 1, 5)

X1, X2, X3, X4, X5, X6, X7, X8  = np.meshgrid(x1, x2, x3, x4, x5, x6, x7, x8)
X_grid = np.vstack((X1.ravel(), X2.ravel(), X3.ravel(), X4.ravel(), X5.ravel(), X6.ravel(), X7.ravel(), X8.ravel())).T

# --- Step 5. Predict mean and std from the GP ---
y_mean, y_std = gp.predict(X_grid, return_std=True)

# --- Step 6. Define the UCB (Upper Confidence Bound) acquisition function ---
beta = 2.0  # exploration parameter
ucb = y_mean + beta * y_std

# --- Step 7. Select next acquisition point ---
next_idx = np.argmax(ucb)
X_next = X_grid[next_idx]
print("Next acquisition point (UCB):", X_next)

format_query(X_next)

ValueError: Anisotropic kernel must have the same number of dimensions as data (6!=8)

In [ ]:
def argmax(x,y):
    idx_ymax = np.argmax(y)
    x_of_ymax = x[idx_ymax]
    return x_of_ymax






In [6]:
for i in range(0, 8):
    print(f"================. FUNCTION {i+1} ===================")
    print(f"x={f_in[i]}")
    print("")
    print(f"y={f_out[i]}")
    print("")
    xmin, xmax, xrange, xmean = xStats(f_in[i])
    print(f"x_min={xmin}")
    print(f"x_max={xmax}")
    print(f"x_range={xrange}")
    print(f"x_mean={xmean}")
    ymin, ymax, yrange, ymean = xStats(f_out[i])
    print(f"y_min={ymin}")
    print(f"y_max={ymax}")
    print(f"y_range={yrange}")
    print(f"y_mean={ymean}")  

    x_of_max_y = argmax(f_in[i],f_out[i])
    print("")
    print(f"x that maximizes y={x_of_max_y}")
    print("")

    print("-----Point to query--------")
    queryPoint = xmean + 3*(x_of_max_y-xmean)
    print(f"queryPoint={queryPoint}")

    # print the elements of the vector queryPoint, in the format 0.000000-0.000000-0.000000
    # where there is one 0.000000 per element in the vector
    query_str=""
    query_str.join(f"{queryPoint[0]:.6f}")
    query_str.join([f"{elem:.6f}" for elem in np.ravel(queryPoint[1:])])
    print(f"queryPoint (formatted)={query_str}")


    









    


================. FUNCTION 1 ===================
x=[[0.31940389 0.76295937]
 [0.57432921 0.8798981 ]
 [0.73102363 0.73299988]
 [0.84035342 0.26473161]
 [0.65011406 0.68152635]
 [0.41043714 0.1475543 ]
 [0.31269116 0.07872278]
 [0.68341817 0.86105746]
 [0.08250725 0.40348751]
 [0.88388983 0.58225397]]

y=[ 1.32267704e-079  1.03307824e-046  7.71087511e-016  3.34177101e-124
 -3.60606264e-003 -2.15924904e-054 -2.08909327e-091  2.53500115e-040
  3.60677119e-081  6.22985647e-048]

x_min=[0.08250725 0.07872278]
x_max=[0.88388983 0.8798981 ]
x_range=[0.80138258 0.80117533]
x_mean=[0.54881678 0.53951913]
y_min=-0.0036060626443634764
y_max=7.710875114502849e-16
y_range=0.0036060626443642474
y_mean=-0.00036060626443627054

x that maximizes y=[0.73102363 0.73299988]

-----Point to query--------
queryPoint=[1.09543734 1.11996136]
queryPoint (formatted)=
================. FUNCTION 2 ===================
x=[[0.66579958 0.12396913]
 [0.87779099 0.7786275 ]
 [0.14269907 0.34900513]
 [0.84527543 0.711120

In [9]:
###### FUNCTION 1 IN-Depth  ######

df_f1 = fcn_as_df(f_in[0],f_out[0])
print(df_f1)
# Week 1 measurement
newRow = fcn_as_df(np.array([0.997304, 1.]),np.float64(3.2158930979511735e-191))
df_f1 = pd.concat([df_f1,newRow])
print(df_f1)




         x1        x2              y
0  0.319404  0.762959   1.322677e-79
1  0.574329  0.879898   1.033078e-46
2  0.731024  0.733000   7.710875e-16
3  0.840353  0.264732  3.341771e-124
4  0.650114  0.681526  -3.606063e-03
5  0.410437  0.147554  -2.159249e-54
6  0.312691  0.078723  -2.089093e-91
7  0.683418  0.861057   2.535001e-40
8  0.082507  0.403488   3.606771e-81
9  0.883890  0.582254   6.229856e-48
         x1        x2              y
0  0.319404  0.762959   1.322677e-79
1  0.574329  0.879898   1.033078e-46
2  0.731024  0.733000   7.710875e-16
3  0.840353  0.264732  3.341771e-124
4  0.650114  0.681526  -3.606063e-03
5  0.410437  0.147554  -2.159249e-54
6  0.312691  0.078723  -2.089093e-91
7  0.683418  0.861057   2.535001e-40
8  0.082507  0.403488   3.606771e-81
9  0.883890  0.582254   6.229856e-48
0  0.997304  1.000000  3.215893e-191


In [ ]:
x = np.array([0.997304, 1.]).reshape(1,-1)
print(x.shape)
#print(np.ravel(np.array([0.997304, 1.])))
#print(np.ravel(np.array([0.997304, 1.]))))
#newRow = fcn_as_df(np.ravel(np.array([0.997304, 1.])),np.float64(3.2158930979511735e-191))

(1, 2)


In [ ]:
# ...existing code...
    # print the elements of the vector queryPoint, in the format 0.000000-0.000000-0.000000
    # where there is one 0.000000 per element in the vector
    query_str = '-'.join([f"{elem:.6f}" for elem in np.ravel(queryPoint)])
    print(f"queryPoint (formatted)={query_str}")
# ...existing code...

In [ ]:
# ...existing code...
    # print the elements of the vector queryPoint, in the format 0.000000-0.000000-0.000000
    # where there is one 0.000000 per element in the vector
    query_str = '-'.join([f"{elem:.6f}" for elem in np.ravel(queryPoint)])
    print(f"queryPoint (formatted)={query_str}")
# ...existing code...